In [ ]:
import os
print(os.listdir("../input"))

Подключаем необходимые библиотеки.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(10, 8)}); 
from sklearn.metrics import mean_squared_error
from scipy.stats import normaltest
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.tree import export_graphviz
from sklearn.ensemble import RandomForestRegressor

# Деревья решений
1.1 Загружаем датасет

In [ ]:
df = pd.read_csv('../input/red-wine-quality-cortez-et-al-2009/winequality-red.csv')
df.head(20).T

In [ ]:
df.info()

In [ ]:
df_target = df['quality']
df_target

В данном датасете собраны данные о свойствах красного вина - всего 12 фич, из которых 11 изначальные, и 1 target - quality. Так как класифицировать здесь нечего - все собранные данные относятся к красному вину - рассматривается задача регрессии. А точнее - какие признаки и в какой степени влияют на целевую переменную.

In [ ]:
sns.kdeplot(df_target)

In [ ]:
df['quality_log'] = np.log(df['quality'])
df_target_log = df['quality_log']
print(df_target)
print(df_target_log)

Для масштабирования воспользуемся классом StandardScaler. Подготовим датафрейм для дальнейшего разделения на выборки - сохраним target-переменную отдельно.

In [ ]:
scaler = StandardScaler()
df_scaled = df.drop('quality', axis = 1)
df2 = df_scaled
df_scaled = df.drop('quality_log', axis = 1)
df3 = df2.drop('quality_log', axis = 1)
df_scaled_fin = scaler.fit_transform(df3)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(df_scaled_fin, df_target_log, test_size=0.25, random_state=412)
df_scaled_fin.shape

In [ ]:
tree = DecisionTreeRegressor(max_depth=3, random_state=412)
tree.fit(X_train, y_train)
tree

In [ ]:
y_pred = tree.predict(X_valid)

print(tree.score(X_valid, y_valid))
print(mean_squared_error(y_valid, y_pred))

# 2. Настройка гиперпараметров дерева
2.1 Создадим генератор разбиений 

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=412)
tree = DecisionTreeRegressor(max_depth=3, random_state=412)

In [ ]:
tree_params = {'max_depth': np.arange(1, 12)}
tree_grid_depth = GridSearchCV(tree, tree_params, scoring='neg_mean_squared_error', cv=kf, n_jobs = -1)
tree_grid_depth.fit(X_train, y_train)

In [ ]:
best_params_depth = tree_grid_depth.best_params_['max_depth']

print(tree_grid_depth.best_estimator_)
print(best_params_depth)
print(tree_grid_depth.best_score_)

In [ ]:
tree_params = {'min_samples_split':np.arange(1,12)}
tree = DecisionTreeRegressor(max_depth = best_params_depth)
tree_grid_samples_split = GridSearchCV(tree, tree_params, scoring='neg_mean_squared_error', cv=kf, n_jobs = -1)
tree_grid_samples_split.fit(X_train, y_train)

In [ ]:
best_params_samples_split = tree_grid_samples_split.best_params_['min_samples_split']

print(tree_grid_samples_split.best_estimator_)
print(best_params_samples_split)
print(tree_grid_samples_split.best_score_)

In [ ]:
tree_params = {'min_samples_leaf':np.arange(1,12)}
tree = DecisionTreeRegressor(max_depth = best_params_depth, min_samples_split = best_params_samples_split)
tree_grid_samples_leaf = GridSearchCV(tree, tree_params, cv=kf, scoring='neg_mean_squared_error', n_jobs = -1)
tree_grid_samples_leaf.fit(X_train, y_train)

In [ ]:
best_params_samples_leaf = tree_grid_samples_leaf.best_params_['min_samples_leaf']

print(tree_grid_samples_leaf.best_estimator_)
print(best_params_samples_leaf)
print(tree_grid_samples_leaf.best_score_)

In [ ]:
tree_params = {'max_features':np.arange(1,12)}
tree = DecisionTreeRegressor(max_depth = best_params_depth, min_samples_split = best_params_samples_split, 
                             min_samples_leaf = best_params_samples_leaf)
tree_grid_max_features = GridSearchCV(tree, tree_params, cv=kf, scoring='neg_mean_squared_error', n_jobs = -1)
tree_grid_max_features.fit(X_train, y_train)

In [ ]:
best_params_max_features = tree_grid_max_features.best_params_['max_features']

print(tree_grid_max_features.best_estimator_)
print(best_params_max_features)
print(tree_grid_max_features.best_score_)

In [ ]:
results_grid_depth = pd.DataFrame(tree_grid_depth.cv_results_)
results_samples_split = pd.DataFrame(tree_grid_samples_split.cv_results_)
results_grid_samples_leaf = pd.DataFrame(tree_grid_samples_leaf.cv_results_)
results_grid_max_features = pd.DataFrame(tree_grid_max_features.cv_results_)

2.2 Постройте валидационные кривые. Сделайте выводы.

In [ ]:
fig, ax = plt.subplots(2, 2, figsize = (10,10))

ax[0,0].set_xlabel("Max depth")
ax[0,0].set_ylabel("Test error")
ax[0,0].plot(results_grid_depth['param_max_depth'], results_grid_depth['mean_test_score']);

ax[0,1].set_xlabel("Min samples split")
ax[0,1].set_ylabel("Test error")
ax[0,1].plot(results_samples_split['param_min_samples_split'], results_samples_split['mean_test_score']);

ax[1,0].set_xlabel("Min samples leaf")
ax[1,0].set_ylabel("Test error")
ax[1,0].plot(results_grid_samples_leaf['param_min_samples_leaf'],results_grid_samples_leaf['mean_test_score']);

ax[1,1].set_xlabel("Max features")
ax[1,1].set_ylabel("Test error")
ax[1,1].plot(results_grid_max_features['param_max_features'], results_grid_max_features['mean_test_score']);

In [ ]:
print('Один из вариантов гиперпараметров:')
print('best_params_depth = ', best_params_depth)
print('best_params_samples_split = ', best_params_samples_split)
print('best_params_samples_leaf = ', best_params_samples_leaf)
print('best_params_max_features = ', best_params_max_features)

2.4 Постройте графически полученное дерево. Оцените важность признаков

In [ ]:
tree = DecisionTreeRegressor(max_depth = best_params_depth, min_samples_split = best_params_samples_split, min_samples_leaf = best_params_samples_leaf, 
                                  max_features =  best_params_max_features, random_state = 412)
tree.fit(X_train, y_train)
y_pred = tree.predict(X_valid)
mean_squared_error(y_valid, y_pred)

In [ ]:
export_graphviz(tree, out_file = 'tree.dot', feature_names = df3.columns)
print(open('tree.dot').read())

Один из вариантов отрисовки дерева:

![](data:image/svg+xml;charset=utf-8,%3C%3Fxml%20version%3D%221.0%22%20encoding%3D%22UTF-8%22%20standalone%3D%22no%22%3F%3E%3C!DOCTYPE%20svg%20PUBLIC%20%22-%2F%2FW3C%2F%2FDTD%20SVG%201.1%2F%2FEN%22%20%22http%3A%2F%2Fwww.w3.org%2FGraphics%2FSVG%2F1.1%2FDTD%2Fsvg11.dtd%22%3E%3C!--%20Generated%20by%20graphviz%20version%202.40.1%20(20161225.0304)%0A%20--%3E%3C!--%20Title%3A%20Tree%20Pages%3A%201%20--%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20xmlns%3Axlink%3D%22http%3A%2F%2Fwww.w3.org%2F1999%2Fxlink%22%20width%3D%221778pt%22%20height%3D%22511pt%22%20viewBox%3D%220.00%200.00%201777.85%20511.20%22%3E%0A%3Cg%20id%3D%22graph0%22%20class%3D%22graph%22%20transform%3D%22scale(1%201)%20rotate(0)%20translate(4%20507.2)%22%3E%0A%3Ctitle%3ETree%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22%23ffffff%22%20stroke%3D%22transparent%22%20points%3D%22-4%2C4%20-4%2C-507.2%201773.8543%2C-507.2%201773.8543%2C4%20-4%2C4%22%2F%3E%0A%3C!--%200%20--%3E%0A%3Cg%20id%3D%22node1%22%20class%3D%22node%22%3E%0A%3Ctitle%3E0%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22942.0465%2C-503.3003%20814.9477%2C-503.3003%20814.9477%2C-427.8997%20942.0465%2C-427.8997%20942.0465%2C-503.3003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22878.4971%22%20y%3D%22-486.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esulphates%20%26lt%3B%3D%20-0.078%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22878.4971%22%20y%3D%22-469.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Emse%20%3D%200.021%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22878.4971%22%20y%3D%22-453%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%201199%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22878.4971%22%20y%3D%22-436.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%201.717%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%201%20--%3E%0A%3Cg%20id%3D%22node2%22%20class%3D%22node%22%3E%0A%3Ctitle%3E1%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22739.4978%2C-392.1003%20627.4964%2C-392.1003%20627.4964%2C-316.6997%20739.4978%2C-316.6997%20739.4978%2C-392.1003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22683.4971%22%20y%3D%22-375.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Ealcohol%20%26lt%3B%3D%200.964%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22683.4971%22%20y%3D%22-358.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Emse%20%3D%200.018%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22683.4971%22%20y%3D%22-341.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20691%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22683.4971%22%20y%3D%22-325%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%201.674%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%200%26%2345%3B%26gt%3B1%20--%3E%0A%3Cg%20id%3D%22edge1%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E0-%26gt%3B1%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M814.9166%2C-429.3428C793.5239%2C-417.1435%20769.6424%2C-403.5249%20748.1668%2C-391.2783%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22749.8607%2C-388.2152%20739.44%2C-386.3018%20746.393%2C-394.296%20749.8607%2C-388.2152%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22746.0393%22%20y%3D%22-406.2151%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3ETrue%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2016%20--%3E%0A%3Cg%20id%3D%22node17%22%20class%3D%22node%22%3E%0A%3Ctitle%3E16%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221168.4978%2C-392.1003%201056.4964%2C-392.1003%201056.4964%2C-316.6997%201168.4978%2C-316.6997%201168.4978%2C-392.1003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221112.4971%22%20y%3D%22-375.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Ealcohol%20%26lt%3B%3D%200.119%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221112.4971%22%20y%3D%22-358.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Emse%20%3D%200.019%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221112.4971%22%20y%3D%22-341.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20508%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221112.4971%22%20y%3D%22-325%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%201.776%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%200%26%2345%3B%26gt%3B16%20--%3E%0A%3Cg%20id%3D%22edge16%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E0-%26gt%3B16%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M942.1295%2C-435.361C974.7878%2C-419.8413%201014.5063%2C-400.9665%201047.3754%2C-385.3467%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%221048.9232%2C-388.4864%201056.4529%2C-381.033%201045.9186%2C-382.1639%201048.9232%2C-388.4864%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221048.0739%22%20y%3D%22-400.387%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3EFalse%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%202%20--%3E%0A%3Cg%20id%3D%22node3%22%20class%3D%22node%22%3E%0A%3Ctitle%3E2%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22436.3789%2C-280.9003%20282.6153%2C-280.9003%20282.6153%2C-205.4997%20436.3789%2C-205.4997%20436.3789%2C-280.9003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22359.4971%22%20y%3D%22-264.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evolatile%20acidity%20%26lt%3B%3D%201.926%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22359.4971%22%20y%3D%22-247.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Emse%20%3D%200.017%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22359.4971%22%20y%3D%22-230.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20589%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22359.4971%22%20y%3D%22-213.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%201.656%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%201%26%2345%3B%26gt%3B2%20--%3E%0A%3Cg%20id%3D%22edge2%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E1-%26gt%3B2%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M627.4305%2C-335.1574C577.5216%2C-318.0281%20503.7251%2C-292.7005%20446.2104%2C-272.9608%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22446.9943%2C-269.5296%20436.3997%2C-269.5937%20444.7219%2C-276.1505%20446.9943%2C-269.5296%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%209%20--%3E%0A%3Cg%20id%3D%22node10%22%20class%3D%22node%22%3E%0A%3Ctitle%3E9%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22746.7628%2C-280.9003%20620.2314%2C-280.9003%20620.2314%2C-205.4997%20746.7628%2C-205.4997%20746.7628%2C-280.9003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22683.4971%22%20y%3D%22-264.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Echlorides%20%26lt%3B%3D%20-0.637%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22683.4971%22%20y%3D%22-247.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Emse%20%3D%200.017%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22683.4971%22%20y%3D%22-230.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20102%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22683.4971%22%20y%3D%22-213.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%201.777%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%201%26%2345%3B%26gt%3B9%20--%3E%0A%3Cg%20id%3D%22edge9%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E1-%26gt%3B9%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M683.4971%2C-316.7528C683.4971%2C-308.42%20683.4971%2C-299.4845%20683.4971%2C-290.8137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22686.9972%2C-290.8084%20683.4971%2C-280.8085%20679.9972%2C-290.8085%20686.9972%2C-290.8084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%203%20--%3E%0A%3Cg%20id%3D%22node4%22%20class%3D%22node%22%3E%0A%3Ctitle%3E3%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22221.6574%2C-169.7003%20105.3368%2C-169.7003%20105.3368%2C-94.2997%20221.6574%2C-94.2997%20221.6574%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22163.4971%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Ealcohol%20%26lt%3B%3D%20-0.256%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22163.4971%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Emse%20%3D%200.014%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22163.4971%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20548%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22163.4971%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%201.665%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%202%26%2345%3B%26gt%3B3%20--%3E%0A%3Cg%20id%3D%22edge3%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E2-%26gt%3B3%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M293.1404%2C-205.5528C272.9216%2C-194.0816%20250.6897%2C-181.4684%20230.4328%2C-169.9758%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22232.0073%2C-166.845%20221.5825%2C-164.9546%20228.5531%2C-172.9334%20232.0073%2C-166.845%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%206%20--%3E%0A%3Cg%20id%3D%22node7%22%20class%3D%22node%22%3E%0A%3Ctitle%3E6%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22450.4862%2C-169.7003%20268.508%2C-169.7003%20268.508%2C-94.2997%20450.4862%2C-94.2997%20450.4862%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22359.4971%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Etotal%20sulfur%20dioxide%20%26lt%3B%3D%20-0.607%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22359.4971%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Emse%20%3D%200.04%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22359.4971%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2041%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22359.4971%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%201.535%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%202%26%2345%3B%26gt%3B6%20--%3E%0A%3Cg%20id%3D%22edge6%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E2-%26gt%3B6%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M359.4971%2C-205.5528C359.4971%2C-197.22%20359.4971%2C-188.2845%20359.4971%2C-179.6137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22362.9972%2C-179.6084%20359.4971%2C-169.6085%20355.9972%2C-179.6085%20362.9972%2C-179.6084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%204%20--%3E%0A%3Cg%20id%3D%22node5%22%20class%3D%22node%22%3E%0A%3Ctitle%3E4%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%2296.9913%2C-58.6014%20.0029%2C-58.6014%20.0029%2C.2014%2096.9913%2C.2014%2096.9913%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%2248.4971%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Emse%20%3D%200.012%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%2248.4971%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20387%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%2248.4971%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%201.646%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%203%26%2345%3B%26gt%3B4%20--%3E%0A%3Cg%20id%3D%22edge4%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E3-%26gt%3B4%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M121.3283%2C-94.3048C110.7022%2C-84.806%2099.3225%2C-74.6335%2088.7909%2C-65.2192%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%2291.0833%2C-62.5738%2081.2952%2C-58.5186%2086.4181%2C-67.7926%2091.0833%2C-62.5738%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%205%20--%3E%0A%3Cg%20id%3D%22node6%22%20class%3D%22node%22%3E%0A%3Ctitle%3E5%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22211.9913%2C-58.6014%20115.0029%2C-58.6014%20115.0029%2C.2014%20211.9913%2C.2014%20211.9913%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22163.4971%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Emse%20%3D%200.016%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22163.4971%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20161%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22163.4971%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%201.709%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%203%26%2345%3B%26gt%3B5%20--%3E%0A%3Cg%20id%3D%22edge5%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E3-%26gt%3B5%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M163.4971%2C-94.3048C163.4971%2C-85.9126%20163.4971%2C-76.9946%20163.4971%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22166.9972%2C-68.5186%20163.4971%2C-58.5186%20159.9972%2C-68.5187%20166.9972%2C-68.5186%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%207%20--%3E%0A%3Cg%20id%3D%22node8%22%20class%3D%22node%22%3E%0A%3Ctitle%3E7%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22322.7119%2C-58.6014%20230.2823%2C-58.6014%20230.2823%2C.2014%20322.7119%2C.2014%20322.7119%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22276.4971%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Emse%20%3D%200.042%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22276.4971%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2019%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22276.4971%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%201.466%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%206%26%2345%3B%26gt%3B7%20--%3E%0A%3Cg%20id%3D%22edge7%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E6-%26gt%3B7%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M329.0623%2C-94.3048C321.6908%2C-85.1748%20313.8169%2C-75.4225%20306.467%2C-66.3193%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22309.174%2C-64.1005%20300.1688%2C-58.5186%20303.7276%2C-68.4979%20309.174%2C-64.1005%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%208%20--%3E%0A%3Cg%20id%3D%22node9%22%20class%3D%22node%22%3E%0A%3Ctitle%3E8%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22433.7119%2C-58.6014%20341.2823%2C-58.6014%20341.2823%2C.2014%20433.7119%2C.2014%20433.7119%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22387.4971%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Emse%20%3D%200.03%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22387.4971%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2022%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22387.4971%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%201.595%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%206%26%2345%3B%26gt%3B8%20--%3E%0A%3Cg%20id%3D%22edge8%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E6-%26gt%3B8%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M369.7643%2C-94.3048C372.0752%2C-85.8204%20374.5325%2C-76.7985%20376.8583%2C-68.2596%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22380.2604%2C-69.087%20379.5115%2C-58.5186%20373.5064%2C-67.2473%20380.2604%2C-69.087%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2010%20--%3E%0A%3Cg%20id%3D%22node11%22%20class%3D%22node%22%3E%0A%3Ctitle%3E10%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22686.8498%2C-169.7003%20508.1444%2C-169.7003%20508.1444%2C-94.2997%20686.8498%2C-94.2997%20686.8498%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22597.4971%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Efree%20sulfur%20dioxide%20%26lt%3B%3D%20-0.179%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22597.4971%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Emse%20%3D%200.027%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22597.4971%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2025%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22597.4971%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%201.695%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%209%26%2345%3B%26gt%3B10%20--%3E%0A%3Cg%20id%3D%22edge10%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E9-%26gt%3B10%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M654.3814%2C-205.5528C647.4467%2C-196.586%20639.9722%2C-186.9213%20632.7938%2C-177.6395%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22635.4692%2C-175.3776%20626.5828%2C-169.6085%20629.9319%2C-179.6601%20635.4692%2C-175.3776%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2013%20--%3E%0A%3Cg%20id%3D%22node14%22%20class%3D%22node%22%3E%0A%3Ctitle%3E13%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22834.5291%2C-169.7003%20704.4651%2C-169.7003%20704.4651%2C-94.2997%20834.5291%2C-94.2997%20834.5291%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22769.4971%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Ecitric%20acid%20%26lt%3B%3D%20-0.698%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22769.4971%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Emse%20%3D%200.01%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22769.4971%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2077%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22769.4971%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%201.804%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%209%26%2345%3B%26gt%3B13%20--%3E%0A%3Cg%20id%3D%22edge13%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E9-%26gt%3B13%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M712.6128%2C-205.5528C719.5475%2C-196.586%20727.022%2C-186.9213%20734.2004%2C-177.6395%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22737.0623%2C-179.6601%20740.4114%2C-169.6085%20731.525%2C-175.3776%20737.0623%2C-179.6601%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2011%20--%3E%0A%3Cg%20id%3D%22node12%22%20class%3D%22node%22%3E%0A%3Ctitle%3E11%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22544.7119%2C-58.6014%20452.2823%2C-58.6014%20452.2823%2C.2014%20544.7119%2C.2014%20544.7119%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22498.4971%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Emse%20%3D%200.03%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22498.4971%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2011%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22498.4971%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%201.595%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2010%26%2345%3B%26gt%3B11%20--%3E%0A%3Cg%20id%3D%22edge11%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E10-%26gt%3B11%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M561.1953%2C-94.3048C552.2252%2C-84.9904%20542.6313%2C-75.0283%20533.7138%2C-65.7684%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22536.1897%2C-63.2938%20526.732%2C-58.5186%20531.1476%2C-68.1495%20536.1897%2C-63.2938%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2012%20--%3E%0A%3Cg%20id%3D%22node13%22%20class%3D%22node%22%3E%0A%3Ctitle%3E12%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22655.7119%2C-58.6014%20563.2823%2C-58.6014%20563.2823%2C.2014%20655.7119%2C.2014%20655.7119%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22609.4971%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Emse%20%3D%200.01%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22609.4971%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2014%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22609.4971%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%201.775%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2010%26%2345%3B%26gt%3B12%20--%3E%0A%3Cg%20id%3D%22edge12%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E10-%26gt%3B12%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M601.8973%2C-94.3048C602.877%2C-85.9126%20603.918%2C-76.9946%20604.9051%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22608.3916%2C-68.857%20606.0747%2C-58.5186%20601.4388%2C-68.0454%20608.3916%2C-68.857%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2014%20--%3E%0A%3Cg%20id%3D%22node15%22%20class%3D%22node%22%3E%0A%3Ctitle%3E14%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22766.7119%2C-58.6014%20674.2823%2C-58.6014%20674.2823%2C.2014%20766.7119%2C.2014%20766.7119%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22720.4971%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Emse%20%3D%200.014%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22720.4971%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2031%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22720.4971%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%201.763%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2013%26%2345%3B%26gt%3B14%20--%3E%0A%3Cg%20id%3D%22edge14%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E13-%26gt%3B14%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M751.5295%2C-94.3048C747.3975%2C-85.636%20742.998%2C-76.406%20738.8499%2C-67.7034%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22741.9342%2C-66.0396%20734.4719%2C-58.5186%20735.6153%2C-69.0516%20741.9342%2C-66.0396%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2015%20--%3E%0A%3Cg%20id%3D%22node16%22%20class%3D%22node%22%3E%0A%3Ctitle%3E15%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22877.7119%2C-58.6014%20785.2823%2C-58.6014%20785.2823%2C.2014%20877.7119%2C.2014%20877.7119%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22831.4971%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Emse%20%3D%200.006%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22831.4971%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2046%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22831.4971%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%201.831%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2013%26%2345%3B%26gt%3B15%20--%3E%0A%3Cg%20id%3D%22edge15%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E13-%26gt%3B15%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M792.2316%2C-94.3048C797.5711%2C-85.4515%20803.2636%2C-76.013%20808.6098%2C-67.1487%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22811.6472%2C-68.8894%20813.8147%2C-58.5186%20805.653%2C-65.2742%20811.6472%2C-68.8894%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2017%20--%3E%0A%3Cg%20id%3D%22node18%22%20class%3D%22node%22%3E%0A%3Ctitle%3E17%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221200.825%2C-280.9003%201024.1692%2C-280.9003%201024.1692%2C-205.4997%201200.825%2C-205.4997%201200.825%2C-280.9003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221112.4971%22%20y%3D%22-264.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Etotal%20sulfur%20dioxide%20%26lt%3B%3D%200.031%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221112.4971%22%20y%3D%22-247.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Emse%20%3D%200.017%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221112.4971%22%20y%3D%22-230.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20251%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221112.4971%22%20y%3D%22-213.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%201.711%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2016%26%2345%3B%26gt%3B17%20--%3E%0A%3Cg%20id%3D%22edge17%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E16-%26gt%3B17%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M1112.4971%2C-316.7528C1112.4971%2C-308.42%201112.4971%2C-299.4845%201112.4971%2C-290.8137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%221115.9972%2C-290.8084%201112.4971%2C-280.8085%201108.9972%2C-290.8085%201115.9972%2C-290.8084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2024%20--%3E%0A%3Cg%20id%3D%22node25%22%20class%3D%22node%22%3E%0A%3Ctitle%3E24%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221522.1892%2C-280.9003%201348.805%2C-280.9003%201348.805%2C-205.4997%201522.1892%2C-205.4997%201522.1892%2C-280.9003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221435.4971%22%20y%3D%22-264.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Efree%20sulfur%20dioxide%20%26lt%3B%3D%200.347%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221435.4971%22%20y%3D%22-247.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Emse%20%3D%200.014%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221435.4971%22%20y%3D%22-230.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20257%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221435.4971%22%20y%3D%22-213.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%201.84%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2016%26%2345%3B%26gt%3B24%20--%3E%0A%3Cg%20id%3D%22edge24%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E16-%26gt%3B24%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M1168.3907%2C-335.1574C1215.3292%2C-318.9977%201283.462%2C-295.5415%201339.1662%2C-276.3641%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%221340.3866%2C-279.6456%201348.7026%2C-273.081%201338.1079%2C-273.0269%201340.3866%2C-279.6456%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2018%20--%3E%0A%3Cg%20id%3D%22node19%22%20class%3D%22node%22%3E%0A%3Ctitle%3E18%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221098.6535%2C-169.7003%20958.3407%2C-169.7003%20958.3407%2C-94.2997%201098.6535%2C-94.2997%201098.6535%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221028.4971%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Efixed%20acidity%20%26lt%3B%3D%201.856%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221028.4971%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Emse%20%3D%200.014%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221028.4971%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20143%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221028.4971%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%201.749%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2017%26%2345%3B%26gt%3B18%20--%3E%0A%3Cg%20id%3D%22edge18%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E17-%26gt%3B18%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M1084.0585%2C-205.5528C1077.2851%2C-196.586%201069.9844%2C-186.9213%201062.973%2C-177.6395%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%221065.7267%2C-175.4781%201056.9064%2C-169.6085%201060.1412%2C-179.6974%201065.7267%2C-175.4781%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2021%20--%3E%0A%3Cg%20id%3D%22node22%22%20class%3D%22node%22%3E%0A%3Ctitle%3E21%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221275.5402%2C-169.7003%201117.454%2C-169.7003%201117.454%2C-94.2997%201275.5402%2C-94.2997%201275.5402%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221196.4971%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evolatile%20acidity%20%26lt%3B%3D%20-0.574%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221196.4971%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Emse%20%3D%200.015%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221196.4971%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20108%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221196.4971%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%201.661%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2017%26%2345%3B%26gt%3B21%20--%3E%0A%3Cg%20id%3D%22edge21%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E17-%26gt%3B21%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M1140.9357%2C-205.5528C1147.7091%2C-196.586%201155.0098%2C-186.9213%201162.0212%2C-177.6395%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%221164.853%2C-179.6974%201168.0878%2C-169.6085%201159.2675%2C-175.4781%201164.853%2C-179.6974%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2019%20--%3E%0A%3Cg%20id%3D%22node20%22%20class%3D%22node%22%3E%0A%3Ctitle%3E19%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%22992.9913%2C-58.6014%20896.0029%2C-58.6014%20896.0029%2C.2014%20992.9913%2C.2014%20992.9913%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22944.4971%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Emse%20%3D%200.013%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22944.4971%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20125%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22944.4971%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%201.731%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2018%26%2345%3B%26gt%3B19%20--%3E%0A%3Cg%20id%3D%22edge19%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E18-%26gt%3B19%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M997.6956%2C-94.3048C990.2353%2C-85.1748%20982.2665%2C-75.4225%20974.8281%2C-66.3193%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%22977.4917%2C-64.0476%20968.454%2C-58.5186%20972.0712%2C-68.4769%20977.4917%2C-64.0476%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2020%20--%3E%0A%3Cg%20id%3D%22node21%22%20class%3D%22node%22%3E%0A%3Ctitle%3E20%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221103.7119%2C-58.6014%201011.2823%2C-58.6014%201011.2823%2C.2014%201103.7119%2C.2014%201103.7119%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221057.4971%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Emse%20%3D%200.008%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221057.4971%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2018%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221057.4971%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%201.876%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2018%26%2345%3B%26gt%3B20%20--%3E%0A%3Cg%20id%3D%22edge20%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E18-%26gt%3B20%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M1039.131%2C-94.3048C1041.5244%2C-85.8204%201044.0695%2C-76.7985%201046.4783%2C-68.2596%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%221049.8797%2C-69.0933%201049.2263%2C-58.5186%201043.1426%2C-67.1927%201049.8797%2C-69.0933%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2022%20--%3E%0A%3Cg%20id%3D%22node23%22%20class%3D%22node%22%3E%0A%3Ctitle%3E22%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221214.7119%2C-58.6014%201122.2823%2C-58.6014%201122.2823%2C.2014%201214.7119%2C.2014%201214.7119%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221168.4971%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Emse%20%3D%200.014%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221168.4971%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2033%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221168.4971%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%201.723%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2021%26%2345%3B%26gt%3B22%20--%3E%0A%3Cg%20id%3D%22edge22%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E21-%26gt%3B22%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M1186.2299%2C-94.3048C1183.919%2C-85.8204%201181.4617%2C-76.7985%201179.1359%2C-68.2596%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%221182.4878%2C-67.2473%201176.4827%2C-58.5186%201175.7338%2C-69.087%201182.4878%2C-67.2473%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2023%20--%3E%0A%3Cg%20id%3D%22node24%22%20class%3D%22node%22%3E%0A%3Ctitle%3E23%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221325.7119%2C-58.6014%201233.2823%2C-58.6014%201233.2823%2C.2014%201325.7119%2C.2014%201325.7119%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221279.4971%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Emse%20%3D%200.014%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221279.4971%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2075%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221279.4971%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%201.634%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2021%26%2345%3B%26gt%3B23%20--%3E%0A%3Cg%20id%3D%22edge23%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E21-%26gt%3B23%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M1226.9319%2C-94.3048C1234.3034%2C-85.1748%201242.1773%2C-75.4225%201249.5272%2C-66.3193%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%221252.2666%2C-68.4979%201255.8254%2C-58.5186%201246.8202%2C-64.1005%201252.2666%2C-68.4979%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2025%20--%3E%0A%3Cg%20id%3D%22node26%22%20class%3D%22node%22%3E%0A%3Ctitle%3E25%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221491.4978%2C-169.7003%201379.4964%2C-169.7003%201379.4964%2C-94.2997%201491.4978%2C-94.2997%201491.4978%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221435.4971%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Ealcohol%20%26lt%3B%3D%201.058%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221435.4971%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Emse%20%3D%200.012%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221435.4971%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%20170%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221435.4971%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%201.859%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2024%26%2345%3B%26gt%3B25%20--%3E%0A%3Cg%20id%3D%22edge25%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E24-%26gt%3B25%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M1435.4971%2C-205.5528C1435.4971%2C-197.22%201435.4971%2C-188.2845%201435.4971%2C-179.6137%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%221438.9972%2C-179.6084%201435.4971%2C-169.6085%201431.9972%2C-179.6085%201438.9972%2C-179.6084%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2028%20--%3E%0A%3Cg%20id%3D%22node29%22%20class%3D%22node%22%3E%0A%3Ctitle%3E28%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221670.388%2C-169.7003%201554.6062%2C-169.7003%201554.6062%2C-94.2997%201670.388%2C-94.2997%201670.388%2C-169.7003%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221612.4971%22%20y%3D%22-153%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Edensity%20%26lt%3B%3D%20-0.727%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221612.4971%22%20y%3D%22-136.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Emse%20%3D%200.014%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221612.4971%22%20y%3D%22-119.4%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2087%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221612.4971%22%20y%3D%22-102.6%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%201.802%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2024%26%2345%3B%26gt%3B28%20--%3E%0A%3Cg%20id%3D%22edge28%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E24-%26gt%3B28%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M1495.4212%2C-205.5528C1511.7334%2C-195.3046%201529.4966%2C-184.145%201546.1426%2C-173.6871%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%221548.1264%2C-176.5743%201554.732%2C-168.2908%201544.4025%2C-170.647%201548.1264%2C-176.5743%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2026%20--%3E%0A%3Cg%20id%3D%22node27%22%20class%3D%22node%22%3E%0A%3Ctitle%3E26%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221436.7119%2C-58.6014%201344.2823%2C-58.6014%201344.2823%2C.2014%201436.7119%2C.2014%201436.7119%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221390.4971%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Emse%20%3D%200.012%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221390.4971%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2091%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221390.4971%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%201.822%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2025%26%2345%3B%26gt%3B26%20--%3E%0A%3Cg%20id%3D%22edge26%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E25-%26gt%3B26%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M1418.9963%2C-94.3048C1415.2016%2C-85.636%201411.1612%2C-76.406%201407.3517%2C-67.7034%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%221410.5475%2C-66.2759%201403.3311%2C-58.5186%201404.135%2C-69.0829%201410.5475%2C-66.2759%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2027%20--%3E%0A%3Cg%20id%3D%22node28%22%20class%3D%22node%22%3E%0A%3Ctitle%3E27%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221547.7119%2C-58.6014%201455.2823%2C-58.6014%201455.2823%2C.2014%201547.7119%2C.2014%201547.7119%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221501.4971%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Emse%20%3D%200.009%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221501.4971%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2079%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221501.4971%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%201.903%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2025%26%2345%3B%26gt%3B27%20--%3E%0A%3Cg%20id%3D%22edge27%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E25-%26gt%3B27%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M1459.6983%2C-94.3048C1465.3823%2C-85.4515%201471.4421%2C-76.013%201477.1332%2C-67.1487%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%221480.2164%2C-68.8246%201482.6739%2C-58.5186%201474.326%2C-65.0427%201480.2164%2C-68.8246%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2029%20--%3E%0A%3Cg%20id%3D%22node30%22%20class%3D%22node%22%3E%0A%3Ctitle%3E29%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221658.7119%2C-58.6014%201566.2823%2C-58.6014%201566.2823%2C.2014%201658.7119%2C.2014%201658.7119%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221612.4971%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Emse%20%3D%200.01%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221612.4971%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2030%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221612.4971%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%201.861%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2028%26%2345%3B%26gt%3B29%20--%3E%0A%3Cg%20id%3D%22edge29%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E28-%26gt%3B29%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M1612.4971%2C-94.3048C1612.4971%2C-85.9126%201612.4971%2C-76.9946%201612.4971%2C-68.5382%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%221615.9972%2C-68.5186%201612.4971%2C-58.5186%201608.9972%2C-68.5187%201615.9972%2C-68.5186%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%2030%20--%3E%0A%3Cg%20id%3D%22node31%22%20class%3D%22node%22%3E%0A%3Ctitle%3E30%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20points%3D%221769.7119%2C-58.6014%201677.2823%2C-58.6014%201677.2823%2C.2014%201769.7119%2C.2014%201769.7119%2C-58.6014%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221723.4971%22%20y%3D%22-41.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Emse%20%3D%200.012%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221723.4971%22%20y%3D%22-25%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Esamples%20%3D%2057%3C%2Ftext%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221723.4971%22%20y%3D%22-8.2%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%20fill%3D%22%23000000%22%3Evalue%20%3D%201.771%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%2028%26%2345%3B%26gt%3B30%20--%3E%0A%3Cg%20id%3D%22edge30%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E28-%26gt%3B30%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22%23000000%22%20d%3D%22M1653.1991%2C-94.3048C1663.3561%2C-84.8982%201674.2264%2C-74.8309%201684.3085%2C-65.4936%22%2F%3E%0A%3Cpolygon%20fill%3D%22%23000000%22%20stroke%3D%22%23000000%22%20points%3D%221686.8811%2C-67.8814%201691.8398%2C-58.5186%201682.1247%2C-62.7456%201686.8811%2C-67.8814%22%2F%3E%0A%3C%2Fg%3E%0A%3C%2Fg%3E%0A%3C%2Fsvg%3E)

In [ ]:
features = {'f' + str(i + 1):name for (i, name) in zip(range(len(df3.columns)), df3.columns)}
importances = tree.feature_importances_

indices = np.argsort(importances)[:: -1]
num_to_plot = 10
feature_indices = [ind + 1 for ind in indices[:num_to_plot]]

for f in range(num_to_plot):
    print(f + 1, features["f" + str(feature_indices[f])], importances[indices[f]])

In [ ]:
plt.figure(figsize=(15,5))
plt.title("Feature importances")
bars = plt.bar(range(num_to_plot), 
               importances[indices[:num_to_plot]],
               color=([str(i / float(num_to_plot +  1)) for i in range(num_to_plot)]),
               align="center")
ticks = plt.xticks(range(num_to_plot), 
                   feature_indices)
plt.xlim([-1, num_to_plot])
plt.legend(bars, [u''.join(features["f" + str(i)]) for i in feature_indices]);

На качество вина существенно влияют признаки:
* alcohol
* sulphates

Немного влияют:
* volatile acidity
* total sulfur dioxide
* fixed acidity
* density


Все остальные не влияют вообще.

2.5 Сформулировать выводы

Модель, полученная в результате метода так же точна как и в методе ближайших соседей, но вычисляется быстрее. К тому же есть возможность получить наглядную визуализацию.
Наиболее важными факторами для нахождения target-переменной являются alcohol и sulphates.

# 3. Случайный лес
3.1 Постройте модель случайного леса для вашей задачи с гиперпараметрами по умолчанию и оцените качество модели.

In [ ]:
rf = RandomForestRegressor(random_state = 412)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_valid)

mean_squared_error(y_valid, y_pred)

Ошибка немного уменьшилась, а значит качество модели немного улучшилось.

3.2 Осуществите подбор гиперпараметров модели.

In [ ]:
rf_params = {'n_estimators': [250, 300, 350, 400, 450, 500]}
rf_grid_estimators = GridSearchCV(rf, rf_params, cv=kf, scoring='neg_mean_squared_error', n_jobs = -1)
rf_grid_estimators.fit(X_train, y_train)

In [ ]:
best_params_estimators = rf_grid_estimators.best_params_['n_estimators']

print(rf_grid_estimators.best_estimator_)
print(best_params_estimators)
print(rf_grid_estimators.best_score_)

In [ ]:
rf = RandomForestRegressor(random_state = 412, n_estimators = best_params_estimators)
rf_params = {'max_depth': np.arange(1, 20)}
rf_grid_depth = GridSearchCV(rf, rf_params, cv=kf, scoring='neg_mean_squared_error', n_jobs = -1)
rf_grid_depth.fit(X_train, y_train)

In [ ]:
best_params_max_depth = rf_grid_depth.best_params_['max_depth']

print(rf_grid_depth.best_estimator_)
print(best_params_max_depth)
print(rf_grid_depth.best_score_)

In [ ]:
rf = RandomForestRegressor(random_state = 412, n_estimators = best_params_estimators, max_depth = best_params_max_depth)
rf_params = {'min_samples_split': np.arange(1, 12)}
rf_grid_split = GridSearchCV(rf, rf_params, cv=kf, scoring='neg_mean_squared_error', n_jobs = -1)
rf_grid_split.fit(X_train, y_train)

In [ ]:
best_params_min_samples_split = rf_grid_split.best_params_['min_samples_split']

print(rf_grid_split.best_estimator_)
print(best_params_min_samples_split)
print(rf_grid_split.best_score_)

In [ ]:
rf = RandomForestRegressor(random_state = 412, n_estimators = best_params_estimators, max_depth = best_params_max_depth, min_samples_split = best_params_min_samples_split)
rf_params = {'min_samples_leaf': np.arange(1, 10)}
rf_grid_leaf = GridSearchCV(rf, rf_params, cv=kf, scoring='neg_mean_squared_error', n_jobs = -1)
rf_grid_leaf.fit(X_train, y_train)

In [ ]:
best_params_min_samples_leaf = rf_grid_leaf.best_params_['min_samples_leaf']

print(rf_grid_leaf.best_estimator_)
print(best_params_min_samples_leaf)
print(rf_grid_leaf.best_score_)

In [ ]:
rf = RandomForestRegressor(random_state = 412, n_estimators = best_params_estimators, max_depth = best_params_max_depth, min_samples_split = best_params_min_samples_split, 
                           min_samples_leaf = best_params_min_samples_leaf)
rf_params = {'max_features': np.arange(1, 12)}
rf_grid_features = GridSearchCV(rf, rf_params, cv=kf, scoring='neg_mean_squared_error', n_jobs = -1)
rf_grid_features.fit(X_train, y_train)

In [ ]:
best_params_max_features = rf_grid_features.best_params_['max_features']

print(rf_grid_features.best_estimator_)
print(best_params_max_features)
print(rf_grid_features.best_score_)

In [ ]:
results_rf_grid_estimators = pd.DataFrame(rf_grid_estimators.cv_results_)
results_rf_grid_depth = pd.DataFrame(rf_grid_depth.cv_results_)
results_rf_grid_split = pd.DataFrame(rf_grid_split.cv_results_)
results_rf_grid_leaf = pd.DataFrame(rf_grid_leaf.cv_results_)
results_rf_grid_features = pd.DataFrame(rf_grid_features.cv_results_)

3.3 Постройте валидационные кривые для каждого из гиперпараметров.

In [ ]:
fig, ax = plt.subplots(2, 3, figsize = (15,15))

ax[0,0].set_xlabel("N Estimators")
ax[0,0].set_ylabel("Test error")
ax[0,0].plot(results_rf_grid_estimators['param_n_estimators'], results_rf_grid_estimators["mean_test_score"]);

ax[0,1].set_xlabel("Max depth")
ax[0,1].set_ylabel("Test error")
ax[0,1].plot(results_rf_grid_depth['param_max_depth'], results_rf_grid_depth["mean_test_score"]);

ax[0,2].set_xlabel("Min samples split")
ax[0,2].set_ylabel("Test error")
ax[0,2].plot(results_rf_grid_split['param_min_samples_split'],results_rf_grid_split["mean_test_score"]);

ax[1,0].set_xlabel("Min samples leaf")
ax[1,0].set_ylabel("Test error")
ax[1,0].plot(results_rf_grid_leaf['param_min_samples_leaf'], results_rf_grid_leaf['mean_test_score']);

ax[1,1].set_xlabel("Max features")
ax[1,1].set_ylabel("Test error")
ax[1,1].plot(results_rf_grid_features['param_max_features'], results_rf_grid_features['mean_test_score']);

In [ ]:
print('Один из вариантов гиперпараметров:')
print('best_params_estimators = ', best_params_estimators)
print('best_params_max_depth = ', best_params_max_depth)
print('best_params_min_samples_split = ', best_params_min_samples_split)
print('best_params_min_samples_leaf = ', best_params_min_samples_leaf)
print('best_params_max_features = ', best_params_max_features)

3.4 Оцените важность признаков данной модели. Визуализируйте топ-10 самых полезных признаков.

In [ ]:
features = {'f' + str(i + 1):name for (i, name) in zip(range(len(df3.columns)), df3.columns)}
importances = tree.feature_importances_

indices = np.argsort(importances)[:: -1]
num_to_plot = 10
feature_indices = [ind + 1 for ind in indices[:num_to_plot]]

print("Feature ranking:")
for f in range(num_to_plot):
    print(f + 1, features["f" + str(feature_indices[f])], importances[indices[f]])

In [ ]:
plt.figure(figsize=(15,5))
plt.title("Feature importances")
bars = plt.bar(range(num_to_plot), 
               importances[indices[:num_to_plot]],
               color=([str(i / float(num_to_plot +  1)) for i in range(num_to_plot)]),
               align="center")
ticks = plt.xticks(range(num_to_plot), 
                   feature_indices)
plt.xlim([-1, num_to_plot])
plt.legend(bars, [u''.join(features["f" + str(i)]) for i in feature_indices]);

Самыми важными являются те же признаки, что и для дерева решений.

3.5 Сравните результаты метода ближайших соседей, дерева решений и случайного леса. Сформулируйте выводы.

Метод ближайших соседей для меня оказался самым точным из представленых - он является довольно простым, но дольше всех работает. Также он не совсем интуитивно понятен. Наименьший квадрат ошибки равен = -0.0022425669832474974.
 
Дерево решений работает быстрее чем KNN. Удобная визуализация, возможность определения важности фич. Наименьший квадрат ошибки равен = -0.015521350522246685

Случайный лес чуть точнее чем дерево решений. Позволяет справиться с переобучением, но очень долго работает при большом количестве деревьев (в моем случае ~ 400). Наименьший квадрат ошибки равен = -0.012513222747648928